In [1]:
# Importng needed libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from sklearn.linear_model  import LinearRegression

import time
from sklearn import metrics

c:\Users\Abubakr\Anaconda3\envs\ai\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df= pd.read_csv('climate.csv')
df.head()

,Date Time,p (armb),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [3]:
df =df.drop(['Date Time'], axis=1)
df

,p (armb),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [4]:
def get_sequence(data, seq_len, target_name):

    seq_list = []
    target_list = []

    for i in range(0, data.shape[0] - (seq_len+1), seq_len+1):

        seq = data[i: seq_len + i]
        target = data[target_name][seq_len + i]

        seq_list.append(seq)
        target_list.append(target)

    return np.array(seq_list), np.array(target_list)

x, y = get_sequence(df, seq_len= 6, target_name='T (degC)')

#print(x.shape)
#print(y.shape)
#----------------------------------------------------------------------------------#

#Feature extraction
def get_features(x):
    feature = []
    for i in range(x.shape[0]):
        mean_column_1 = np.mean(x[i, :, 0])
        std_column_2 =np.std(x[i, :, 1])
        min_column_3 =np.min(x[i, :, 2])
        std_column_4 =np.std(x[i, :, 3])
        mode_column_5 =np.min(x[i, :, 4])
        median_column_6 =np.min(x[i, :, 5])
        min_column_7 =np.min(x[i, :, 6])
        max_column_8 =np.min(x[i, :, 7])
        mean_column_9 =np.min(x[i, :, 8])
        mode_column_10=np.min(x[i, :, 9])
        median_column_11=np.min(x[i, :, 10])
        std_column_12=np.min(x[i, :, 11])
        mean_column_13=np.min(x[i, :, 12])
        mean_column_14=np.min(x[i, :, 13])

        feature.append((mean_column_1, std_column_2,min_column_3,std_column_4,mode_column_5,median_column_6,min_column_7,
                        max_column_8,mean_column_9,mode_column_10,median_column_11,std_column_12,mean_column_13,mean_column_14))
        #feature =np.hstack((mean_column_1, std_columns_2))
    return np.array(feature)

nx = get_features(x)
#print(nx.shape)
print(nx.shape)
ndf =pd.DataFrame(nx)
ndf

(60078, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,996.523333,0.177615,264.91,0.189444,93.30,3.21,3.01,0.19,1.88,3.02,1307.75,0.19,0.50,136.10
1,996.566667,0.485023,264.54,0.534010,93.10,3.12,2.90,0.19,1.81,2.91,1305.69,0.16,0.50,118.60
2,996.860000,0.103669,264.45,0.123243,93.30,3.10,2.89,0.19,1.81,2.90,1311.37,0.07,0.25,115.30
3,997.083333,0.243630,263.87,0.283279,92.00,2.97,2.73,0.22,1.71,2.74,1312.78,0.10,0.38,66.16
4,997.468333,0.154164,263.43,0.173469,91.70,2.87,2.64,0.22,1.64,2.64,1316.25,0.09,0.50,60.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60073,1002.318333,0.451900,272.03,0.233411,68.25,5.70,4.10,1.60,2.55,4.09,1274.28,0.27,0.64,198.20
60074,1002.190000,0.242762,271.30,0.287542,70.20,5.40,3.86,1.36,2.40,3.85,1280.70,0.64,1.26,190.60
60075,1001.446667,0.774326,269.75,0.214275,65.42,4.78,3.57,1.12,2.22,3.56,1281.87,0.49,1.04,184.30
60076,1001.063333,0.217000,269.98,0.058595,71.10,4.86,3.58,1.28,2.23,3.58,1286.24,0.24,0.52,9.40


In [5]:
def split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test =split(nx, y)

In [6]:
def tree_regressors():
    tree_regressor = {
    'Linear':        LinearRegression(),
    "Skl GBM":       GradientBoostingRegressor(),
    "XGBoost":       XGBRegressor(),
    "LightGBM":      LGBMRegressor()
    }
    return tree_regressor

reg =tree_regressors()
results = pd.DataFrame({'Model': [], 'MSE': [], 'MAB': [], 'Time': [],})
for model_name, model in reg.items():

    start_time = time.time()
    model.fit(X_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(X_test)
    
    results = results.append({"Model":    model_name,
                              "MSE": metrics.mean_squared_error(y_test, pred),
                              "MAB": metrics.mean_absolute_error(y_test, pred),
                              "Time":     total_time
                              },
                              ignore_index=True)



results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')

print(results_ord)


C:\Users\Abubakr\AppData\Local\Temp\ipykernel_12864\3045964275.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\Abubakr\AppData\Local\Temp\ipykernel_12864\3045964275.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\Abubakr\AppData\Local\Temp\ipykernel_12864\3045964275.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,
C:\Users\Abubakr\AppData\Local\Temp\ipykernel_12864\3045964275.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Mod

      Model       MSE       MAB       Time
1  LightGBM  0.252246  0.347588   0.346379
2   XGBoost  0.254560  0.345442   4.126238
3   Skl GBM  0.269239  0.369581  17.636862
4    Linear  0.340799  0.414507   0.031912


In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from sklearn.linear_model  import LinearRegression

import time
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#----------------------------------------------------------------------------------#
def get_sequence(data, seq_len, target_name):

    seq_list = []
    target_list = []

    for i in range(0, data.shape[0] - (seq_len+1), seq_len+1):

        seq = data[i: seq_len + i]
        target = data[target_name][seq_len + i]

        seq_list.append(seq)
        target_list.append(target)

    return np.array(seq_list), np.array(target_list)

x, y = get_sequence(df, seq_len= 6, target_name='T (degC)')

print(x.shape)
print(y.shape)
#----------------------------------------------------------------------------------#

def get_features(x):
    feature = []
    for i in range(x.shape[0]):
        mean_column = x[i].mean(axis=0)
        feature.append(mean_column)
    return np.array(feature)

nx = get_features(x)
print(nx.shape)
mdf =pd.DataFrame(nx)
mdf

(60078, 6, 14)
(60078,)
(60078, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,996.523333,-8.261667,265.161667,-9.063333,93.883333,3.271667,3.071667,0.200000,1.918333,3.081667,1308.973333,0.468333,0.940000,177.500000
1,996.566667,-8.408333,265.015000,-9.260000,93.516667,3.235000,3.025000,0.208333,1.890000,3.036667,1309.785000,0.320000,0.690000,170.650000
2,996.860000,-8.768333,264.630000,-9.613333,93.550000,3.143333,2.940000,0.201667,1.838333,2.950000,1311.991667,0.178333,0.565000,176.550000
3,997.083333,-9.186667,264.193333,-10.178333,92.433333,3.041667,2.810000,0.231667,1.755000,2.818333,1314.430000,0.230000,0.628333,175.193333
4,997.468333,-9.710000,263.640000,-10.725000,92.216667,2.916667,2.691667,0.228333,1.680000,2.696667,1317.613333,0.335000,0.813333,115.286667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60073,1002.318333,-0.138333,272.841667,-4.888333,70.191667,6.050000,4.241667,1.805000,2.640000,4.231667,1276.853333,0.750000,1.386667,218.566667
60074,1002.190000,-1.300000,271.688333,-5.778333,71.416667,5.553333,3.965000,1.586667,2.463333,3.955000,1282.283333,0.903333,1.516667,216.166667
60075,1001.446667,-2.201667,270.846667,-6.928333,69.988333,5.200000,3.630000,1.571667,2.256667,3.623333,1285.766667,0.910000,1.533333,205.300000
60076,1001.063333,-2.653333,270.426667,-6.970000,72.016667,5.025000,3.616667,1.406667,2.248333,3.611667,1287.406667,0.573333,1.250000,98.366667
